In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R
import os
import sys
import open3d as o3d
import json
from glob import glob
import cv2
from visualization.osdar23_plot_3d_boxes import process_detections

sys.path.append(os.path.join(os.getcwd(), "..", "tools"))

# rgb_highres_center

In [ ]:
# fmt: off
sensors = {
    "rgb_highres_center" : {
        "intrinsic" : np.array([7267.95450880415, 0.0, 2056.049238502414, 0.0, 0.0, 7267.95450880415, 1232.862908875167, 0.0, 0.0, 0.0, 1.0, 0.0]).reshape(3, 4),
        "quaternion" : np.array([-0.00313306, 0.0562995, 0.00482918, 0.998397]),
        "translation" : np.array([0.0801578, -0.332862, 3.50982]).reshape(3, 1),
        "distortion_coeffs" : np.array([-0.0764891, -0.188057, 0.0, 0.0, 1.78279]),
        "width_px" : 4112,
        "height_px" : 2504,
        "color" : np.array([29, 91, 121])/255
    },
    "rgb_highres_left" : {
        "intrinsic" : np.array([7265.09513308939, 0.0, 2099.699693520321, 0.0, 0.0, 7265.09513308939, 1217.709330768128, 0.0, 0.0, 0.0, 1.0, 0.0]).reshape(3, 4),
        "quaternion" : np.array([-0.0121052, 0.0606543, 0.188524, 0.980119]),
        "translation" : np.array([0.0609096, -0.136682, 3.51522]).reshape(3, 1),
        "distortion_coeffs" : np.array([-0.0878118, 0.0196239, 0.0, 0.0, 0.902557]),
        "width_px" : 4112,
        "height_px" : 2504,
        "color" : np.array([239, 98, 98])/255
    },
    "rgb_highres_right" : {
        "intrinsic" : np.array([7265.854580654392, 0.0, 2093.506452810741, 0.0, 0.0, 7265.854580654392, 1228.255759518024, 0.0, 0.0, 0.0, 1.0, 0.0]).reshape(3, 4),
        "quaternion" : np.array([0.0217394, 0.0639103, -0.182324, 0.980918]),
        "translation" : np.array([0.0510683, -0.525541, 3.51628]).reshape(3, 1),
        "distortion_coeffs" : np.array([-0.0655375, -0.329857, 0.0, 0.0, 2.48002]),
        "width_px" : 4112,
        "height_px" : 2504,
        "color" : np.array([243, 170, 96])/255
    },
    "rgb_center" : {
        "intrinsic" : np.array([4609.471892628096, 0.0, 1257.158605934, 0.0, 0.0, 4609.471892628096, 820.0498076210201, 0.0, 0.0, 0.0, 1.0, 0.0]).reshape(3, 4),
        "quaternion" : np.array([0.00193374, -0.00293566, 0.00271082, 0.99999]),
        "translation" : np.array([0.0669458, -0.000911152, 2.05432]).reshape(3, 1),
        "distortion_coeffs" : np.array([-0.0914603, 0.605326, 0.0, 0.0, 0.417134]),
        "width_px" : 2464,
        "height_px" : 1600,
        "color" : np.array([29, 91, 121])/255
    },
    "rgb_left" : {
        "intrinsic" : np.array([4622.041473915607, 0.0, 1233.380196060109, 0.0, 0.0, 4622.041473915607, 843.3909933480334, 0.0, 0.0, 0.0, 1.0, 0.0]).reshape(3, 4),
        "quaternion" : np.array([0.00228435, -0.0104673, 0.173904, 0.984704]),
        "translation" : np.array([0.0298925, 0.186612, 2.05637]).reshape(3, 1),
        "distortion_coeffs" : np.array([-0.0868757, 0.53867, 0.0, 0.0, 1.69009]),
        "width_px" : 2464,
        "height_px" : 1600,
        "color" : np.array([239, 98, 98])/255
    },
    "rgb_right" : {
        "intrinsic" : np.array([4613.465257442901, 0.0, 1230.818284106724, 0.0, 0.0, 4613.465257442901, 783.2495217495479, 0.0, 0.0, 0.0, 1.0, 0.0]).reshape(3, 4),
        "quaternion" : np.array([0.00740182, -0.0113748, -0.184053, 0.982823]),
        "translation" : np.array([0.051346, -0.196979, 2.05803]).reshape(3, 1),
        "distortion_coeffs" : np.array([-0.0888195, 0.52648, 0.0, 0.0, 1.27408]),
        "width_px" : 2464,
        "height_px" : 1600,
        "color" : np.array([243, 170, 96])/255
    }
}
# fmt: on

In [ ]:
projection_matrices = {}
for sensor_name, details in sensors.items():
    intrinsic = details["intrinsic"]
    quaternion = details["quaternion"]
    translation = details["translation"]
    distortion_coeffs = details["distortion_coeffs"]
    width = details["width_px"]
    height = details["height_px"]

    intrinsics_undistorted, _ = cv2.getOptimalNewCameraMatrix(intrinsic[:,:-1], distortion_coeffs, (width,height), 1, (width,height))
    intrinsics_undistorted = np.hstack((intrinsics_undistorted, np.asarray([[0.0], [0.0], [0.0]])))

    rotation = R.from_quat(quaternion)
    coord_system_rot = R.from_euler('zxy', [-90, 0, 90], degrees=True)
    rotation = (rotation * coord_system_rot).as_matrix()

    extrinsic = np.vstack((np.hstack((rotation.T, - rotation.T @ translation)), [0.0,0.0,0.0,1.0]))
    projection = np.matmul(intrinsics_undistorted, extrinsic)
    projection_matrices[sensor_name] = projection

    sensors[sensor_name]["projection"] = projection
    sensors[sensor_name]["extrinsic"] = extrinsic
    
    
for x, y in projection_matrices.items():
    print(f"sensor {x}: \n", y)

# Visualize the scene

In [ ]:
# input_id = "1_calibration_1.1"
input_id = "2_station_berliner_tor_2.1"
# input_id = "3_fire_site_3.1"
frame_idx = 0
camera_location = "rgb_highres_left"
input_folder_path = f"C:\\data\\osdar23\\{input_id}"

input_folder_path_images = os.path.join(input_folder_path, camera_location)
input_folder_path_point_clouds = os.path.join(input_folder_path, "lidar")
input_folder_path_detections = os.path.join(input_folder_path, f"{input_id}_labels.json")

In [ ]:
def add_open3d_axis(vis, points:np.ndarray = np.array([[0,0,0], [5,0,0], [0,5,0], [0,0,5]])):
    axis = o3d.geometry.LineSet()
    axis.points = o3d.utility.Vector3dVector(points)
    axis.lines = o3d.utility.Vector2iVector(np.array([[0, 1], [0, 2], [0, 3]]))
    axis.colors = o3d.utility.Vector3dVector(np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]))
    vis.add_geometry(axis)

def visualize_scene(input_folder_path_point_clouds, input_folder_path_detections, frame_idx, cam_details):
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name="Point Cloud Visualizer", width=1600, height=900)
    vis.get_render_option().background_color = [0.1, 0.1, 0.1]
    vis.get_render_option().point_size = 2.0
    vis.get_render_option().show_coordinate_frame = True

    # fmt: off
    file_path_point_cloud = sorted(glob(os.path.join(input_folder_path_point_clouds, "*")))[frame_idx]
    # fmt: on

    pcd = o3d.io.read_point_cloud(file_path_point_cloud)
    pcd.paint_uniform_color([0.5, 0.5, 0.5])

    label_data = json.load(open(input_folder_path_detections))

    process_detections(label_data, pcd, vis, frame_idx, True)

    vis.add_geometry(pcd)

    # world frame
    add_open3d_axis(vis)

    # ccf = o3d.geometry.TriangleMesh.create_coordinate_frame(size=5.0, origin=[0,0,0])
    # ccf.transform(cam_extrinsics)
    # vis.add_geometry(ccf)

    for sensor_name, details in cam_details.items():
        cam = o3d.camera.PinholeCameraIntrinsic(details["width_px"], details["height_px"], details["intrinsic"][:,:-1])
        cam_vis = o3d.geometry.LineSet.create_camera_visualization(cam, extrinsic=details["extrinsic"], scale=100)
        cam_vis.paint_uniform_color(details["color"])
        vis.add_geometry(cam_vis)


    ctr = vis.get_view_control()
    ctr.set_zoom(0.12)
    ctr.set_front([0.22, 0.141, 0.965])
    ctr.set_lookat([22.964, -0.772, 0.230])
    ctr.set_up([0.969, -0.148, -0.200])
    
    vis.poll_events()
    vis.update_renderer()

    vis.run()
    vis.destroy_window()

In [ ]:
visualize_scene(input_folder_path_point_clouds, input_folder_path_detections, frame_idx, sensors)